<a href="https://colab.research.google.com/github/ANGELYNEDEVSENIOR/ACTIVIDAD4/blob/main/SISTEMAACTUALIZADOBIBLIOTECAipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import sqlite3
import os

# ------------------ Inicialización de la Base de Datos ------------------ #
def inicializar_db_tablas():
    conn = sqlite3.connect("biblioteca.db")
    cursor = conn.cursor()

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS usuarios (
            cedula TEXT PRIMARY KEY,
            nombres TEXT NOT NULL,
            apellidos TEXT NOT NULL
        )
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS editoriales (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            nombre TEXT NOT NULL
        )
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS libros (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            isbn TEXT UNIQUE,
            titulo TEXT NOT NULL,
            autor TEXT NOT NULL,
            editorial_id INTEGER NOT NULL,
            FOREIGN KEY(editorial_id) REFERENCES editoriales(id)
        )
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS prestamos (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            cedula_usuario TEXT NOT NULL,
            isbn_libro TEXT NOT NULL,
            fecha_prestamo TEXT DEFAULT (DATE('now')),
            FOREIGN KEY(cedula_usuario) REFERENCES usuarios(cedula),
            FOREIGN KEY(isbn_libro) REFERENCES libros(isbn)
        )
    """)

    conn.commit()
    conn.close()

# --- Aquí agregamos la función para insertar libros manualmente sin input ---
def agregar_libro_manual(isbn, titulo, autor, editorial_id):
    conn = sqlite3.connect("biblioteca.db")
    cursor = conn.cursor()

    # Validar si la editorial existe
    cursor.execute("SELECT id FROM editoriales WHERE id = ?", (editorial_id,))
    if not cursor.fetchone():
        print(f"❌ La editorial con id {editorial_id} no existe. No se puede agregar el libro.")
        conn.close()
        return

    try:
        cursor.execute("""
            INSERT INTO libros (isbn, titulo, autor, editorial_id)
            VALUES (?, ?, ?, ?)
        """, (isbn, titulo, autor, editorial_id))
        conn.commit()
        print(f"✅ Libro '{titulo}' agregado correctamente.")
    except sqlite3.IntegrityError as e:
        if "UNIQUE constraint failed" in str(e):
            print("❌ Error: Ya existe un libro con ese ISBN.")
        else:
            print(f"❌ Error al agregar el libro: {e}")
    finally:
        conn.close()

# ------------------ Clases de Grafos ------------------ #
class GrafoDirigido:
    def __init__(self):
        self.adyacencia = {}

    def agregar_nodo(self, nodo):
        if nodo not in self.adyacencia:
            self.adyacencia[nodo] = set()

    def agregar_arista(self, origen, destino):
        self.agregar_nodo(origen)
        self.agregar_nodo(destino)
        self.adyacencia[origen].add(destino)

    def mostrar_grafo(self):
        print("\nGrafo dirigido (Usuario → ISBN de libro prestado):")
        if not self.adyacencia:
            print("No hay préstamos registrados.")
            return
        for origen, destinos in self.adyacencia.items():
            for destino in destinos:
                print(f"{origen} → {destino}")

class GrafoUsuarios:
    def __init__(self):
        self.adyacencia = {}

    def agregar_usuario(self, usuario_id):
        if usuario_id not in self.adyacencia:
            self.adyacencia[usuario_id] = set()

    def agregar_relacion(self, usuario1, usuario2):
        self.agregar_usuario(usuario1)
        self.agregar_usuario(usuario2)
        self.adyacencia[usuario1].add(usuario2)

    def construir_red(self, usuarios_libros):
        self.adyacencia.clear()
        for id1, libros1 in usuarios_libros.items():
            for id2, libros2 in usuarios_libros.items():
                if id1 != id2 and set(libros1) & set(libros2):
                    self.agregar_relacion(id1, id2)

    def mostrar_red(self):
        print("\nRed de usuarios con libros en común:")
        if not self.adyacencia:
            print("No hay relaciones entre usuarios.")
            return
        for usuario, relacionados in self.adyacencia.items():
            for otro in relacionados:
                print(f"{usuario} → {otro}")

# ------------------ Árbol Binario de Búsqueda ------------------ #
class NodoLibro:
    def __init__(self, titulo, isbn, autor, editorial):
        self.titulo = titulo
        self.isbn = isbn
        self.autor = autor
        self.editorial = editorial
        self.izquierda = None
        self.derecha = None

class ABBLibros:
    def __init__(self):
        self.raiz = None

    def insertar(self, titulo, isbn, autor, editorial):
        def _insertar(nodo, nuevo):
            if not nodo:
                return nuevo
            if nuevo.titulo.lower() < nodo.titulo.lower():
                nodo.izquierda = _insertar(nodo.izquierda, nuevo)
            else:
                nodo.derecha = _insertar(nodo.derecha, nuevo)
            return nodo

        nuevo = NodoLibro(titulo, isbn, autor, editorial)
        self.raiz = _insertar(self.raiz, nuevo)

    def buscar(self, titulo):
        def _buscar(nodo, titulo):
            if not nodo:
                return None
            if nodo.titulo.lower() == titulo.lower():
                return nodo
            elif titulo.lower() < nodo.titulo.lower():
                return _buscar(nodo.izquierda, titulo)
            else:
                return _buscar(nodo.derecha, titulo)
        return _buscar(self.raiz, titulo)

    def recorrido_inorden(self):
        def _inorden(nodo):
            if nodo:
                _inorden(nodo.izquierda)
                print(f"{nodo.titulo} (ISBN: {nodo.isbn}, Autor: {nodo.autor}, Editorial: {nodo.editorial})")
                _inorden(nodo.derecha)
        _inorden(self.raiz)

# ------------------ Funciones CRUD ------------------ #
def agregar_usuario():
    cedula = input("Cédula: ").strip()
    nombres = input("Nombres: ").strip()
    apellidos = input("Apellidos: ").strip()
    if not cedula or not nombres or not apellidos:
        print("Todos los campos son obligatorios.")
        return
    conn = sqlite3.connect("biblioteca.db")
    cursor = conn.cursor()

    try:
        cursor.execute("INSERT INTO usuarios (cedula, nombres, apellidos) VALUES (?, ?, ?)",
                       (cedula, nombres, apellidos))
        conn.commit()
        print("Usuario agregado exitosamente.")
    except sqlite3.IntegrityError:
        print("Error: ya existe un usuario con esa cédula.")
    finally:
        conn.close()

def agregar_editorial():
    nombre = input("Nombre de la editorial: ").strip()
    if not nombre:
        print("El nombre no puede estar vacío.")
        return
    conn = sqlite3.connect("biblioteca.db")
    cursor = conn.cursor()
    cursor.execute("INSERT INTO editoriales (nombre) VALUES (?)", (nombre,))
    conn.commit()
    conn.close()
    print("Editorial agregada exitosamente.")

def agregar_libro():
    isbn = input("ISBN del libro: ").strip()
    titulo = input("Título del libro: ").strip()
    autor = input("Autor del libro: ").strip()

    if not isbn or not titulo or not autor:
        print("Todos los campos son obligatorios.")
        return

    conn = sqlite3.connect("biblioteca.db")
    cursor = conn.cursor()

    # Mostrar editoriales disponibles
    cursor.execute("SELECT id, nombre FROM editoriales")
    editoriales = cursor.fetchall()

    if not editoriales:
        print("⚠️ No hay editoriales registradas. Debes agregar una primero.")
        conn.close()
        return

    print("Editoriales disponibles:")
    for ed in editoriales:
        print(f"{ed[0]} - {ed[1]}")

    editorial_id = input("ID de la editorial del libro: ").strip()
    if not editorial_id.isdigit():
        print("❌ ID de editorial debe ser un número válido.")
        conn.close()
        return

    editorial_id = int(editorial_id)

    cursor.execute("SELECT id FROM editoriales WHERE id = ?", (editorial_id,))
    if not cursor.fetchone():
        print("❌ ID de editorial no válida.")
        conn.close()
        return

    try:
        cursor.execute("""
            INSERT INTO libros (isbn, titulo, autor, editorial_id)
            VALUES (?, ?, ?, ?)
        """, (isbn, titulo, autor, editorial_id))
        conn.commit()
        print("✅ Libro agregado exitosamente.")
    except sqlite3.IntegrityError as e:
        if "UNIQUE constraint failed" in str(e):
            print("❌ Error: Ya existe un libro con ese ISBN.")
        else:
            print(f"❌ Error al agregar el libro: {e}")
    finally:
        conn.close()


def listar_usuarios():
    conn = sqlite3.connect("biblioteca.db")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM usuarios ORDER BY apellidos")
    usuarios = cursor.fetchall()
    conn.close()

    if not usuarios:
        print("No hay usuarios registrados.")
    else:
        print("\nUsuarios registrados:")
        for user in usuarios:
            print(f"Cédula: {user[0]}, Nombres: {user[1]}, Apellidos: {user[2]}")

def listar_libros():
    conn = sqlite3.connect("biblioteca.db")
    cursor = conn.cursor()
    cursor.execute("""
        SELECT libros.id, libros.isbn, libros.titulo, libros.autor, editoriales.nombre
        FROM libros
        JOIN editoriales ON libros.editorial_id = editoriales.id
        ORDER BY libros.titulo
    """)
    libros = cursor.fetchall()
    conn.close()

    if not libros:
        print("No hay libros registrados.")
    else:
        print("\nLibros registrados:")
        for libro in libros:
            print(f"{libro[0]} - {libro[2]} (ISBN: {libro[1]}, Autor: {libro[3]}, Editorial: {libro[4]})")

def listar_editoriales():
    conn = sqlite3.connect("biblioteca.db")
    cursor = conn.cursor()
    cursor.execute("SELECT id, nombre FROM editoriales ORDER BY nombre")
    editoriales = cursor.fetchall()
    conn.close()

    if not editoriales:
        print("No hay editoriales registradas.")
    else:
        print("\nEditoriales registradas:")
        for ed in editoriales:
            print(f"ID: {ed[0]}, Nombre: {ed[1]}")

def prestar_libro(grafo_prestamos):
    cedula = input("Cédula del usuario: ").strip()
    isbn = input("ISBN del libro a prestar: ").strip()
    if not cedula or not isbn:
        print("Cédula y ISBN son obligatorios.")
        return

    conn = sqlite3.connect("biblioteca.db")
    cursor = conn.cursor()

    cursor.execute("SELECT * FROM usuarios WHERE cedula = ?", (cedula,))
    if not cursor.fetchone():
        print("Usuario no encontrado.")
        conn.close()
        return

    cursor.execute("SELECT * FROM libros WHERE isbn = ?", (isbn,))
    if not cursor.fetchone():
        print("Libro no encontrado.")
        conn.close()
        return

    cursor.execute("SELECT * FROM prestamos WHERE isbn_libro = ?", (isbn,))
    if cursor.fetchone():
        print("El libro ya está prestado.")
        conn.close()
        return

    cursor.execute("INSERT INTO prestamos (cedula_usuario, isbn_libro) VALUES (?, ?)", (cedula, isbn))
    conn.commit()
    conn.close()
    grafo_prestamos.agregar_arista(cedula, isbn)
    print(f"Préstamo registrado: Usuario {cedula} → Libro {isbn}")

def cargar_prestamos(grafo_prestamos):
    conn = sqlite3.connect("biblioteca.db")
    cursor = conn.cursor()
    cursor.execute("SELECT cedula_usuario, isbn_libro FROM prestamos")
    for cedula, isbn in cursor.fetchall():
        grafo_prestamos.agregar_arista(cedula, isbn)
    conn.close()

def obtener_usuarios_libros():
    conn = sqlite3.connect("biblioteca.db")
    cursor = conn.cursor()
    cursor.execute("SELECT cedula, isbn_libro FROM prestamos")
    usuarios_libros = {}
    for cedula, isbn in cursor.fetchall():
        usuarios_libros.setdefault(cedula, []).append(isbn)
    conn.close()
    return usuarios_libros

# ------------------ Menú Principal ------------------ #
def menu():
    grafo_prestamos = GrafoDirigido()
    abb_libros = ABBLibros()

    inicializar_db_tablas()
    cargar_prestamos(grafo_prestamos)

    # Ejemplo: agregar editorial y libro automáticamente (solo para pruebas)
    # agregar_editorial()  # o puedes usar el menú para agregar editoriales
    # agregar_libro_manual("978-1234567890", "El Quijote", "Miguel de Cervantes", 1)

    while True:
        print("\nMenú Biblioteca")
        print("1. Agregar Usuario")
        print("2. Agregar Editorial")
        print("3. Agregar Libro")
        print("4. Listar Usuarios")
        print("5. Listar Editoriales")
        print("6. Listar Libros")
        print("7. Registrar Préstamo")
        print("8. Mostrar Préstamos (Grafo Dirigido)")
        print("9. Mostrar Red de Usuarios con libros comunes")
        print("10. Insertar libros en ABB")
        print("11. Buscar libro en ABB")
        print("12. Mostrar libros ABB (Inorden)")
        print("0. Salir")

        opcion = input("Seleccione una opción: ").strip()

        if opcion == "1":
            agregar_usuario()
        elif opcion == "2":
            agregar_editorial()
        elif opcion == "3":
            agregar_libro()
        elif opcion == "4":
            listar_usuarios()
        elif opcion == "5":
            listar_editoriales()
        elif opcion == "6":
            listar_libros()
        elif opcion == "7":
            prestar_libro(grafo_prestamos)
        elif opcion == "8":
            grafo_prestamos.mostrar_grafo()
        elif opcion == "9":
            usuarios_libros = obtener_usuarios_libros()
            grafo_usuarios = GrafoUsuarios()
            grafo_usuarios.construir_red(usuarios_libros)
            grafo_usuarios.mostrar_red()
        elif opcion == "10":
            # Insertar libros en ABB
            conn = sqlite3.connect("biblioteca.db")
            cursor = conn.cursor()
            cursor.execute("""
                SELECT libros.titulo, libros.isbn, libros.autor, editoriales.nombre
                FROM libros
                JOIN editoriales ON libros.editorial_id = editoriales.id
            """)
            libros = cursor.fetchall()
            conn.close()

            for libro in libros:
                abb_libros.insertar(*libro)
            print("Libros insertados en ABB correctamente.")
        elif opcion == "11":
            titulo_buscar = input("Ingrese el título del libro a buscar: ").strip()
            resultado = abb_libros.buscar(titulo_buscar)
            if resultado:
                print(f"Libro encontrado: {resultado.titulo} (ISBN: {resultado.isbn}, Autor: {resultado.autor}, Editorial: {resultado.editorial})")
            else:
                print("Libro no encontrado en ABB.")
        elif opcion == "12":
            abb_libros.recorrido_inorden()
        elif opcion == "0":
            print("Saliendo del programa...")
            break
        else:
            print("Opción no válida, intente de nuevo.")

if __name__ == "__main__":
    menu()



Menú Biblioteca
1. Agregar Usuario
2. Agregar Editorial
3. Agregar Libro
4. Listar Usuarios
5. Listar Editoriales
6. Listar Libros
7. Registrar Préstamo
8. Mostrar Préstamos (Grafo Dirigido)
9. Mostrar Red de Usuarios con libros comunes
10. Insertar libros en ABB
11. Buscar libro en ABB
12. Mostrar libros ABB (Inorden)
0. Salir
Seleccione una opción: 4

Usuarios registrados:
Cédula: 1235, Nombres: leonardo, Apellidos: barbosa
Cédula: 1234, Nombres: angelyne, Apellidos: gonzalez

Menú Biblioteca
1. Agregar Usuario
2. Agregar Editorial
3. Agregar Libro
4. Listar Usuarios
5. Listar Editoriales
6. Listar Libros
7. Registrar Préstamo
8. Mostrar Préstamos (Grafo Dirigido)
9. Mostrar Red de Usuarios con libros comunes
10. Insertar libros en ABB
11. Buscar libro en ABB
12. Mostrar libros ABB (Inorden)
0. Salir
Seleccione una opción: 5

Editoriales registradas:
ID: 1, Nombre: duendes

Menú Biblioteca
1. Agregar Usuario
2. Agregar Editorial
3. Agregar Libro
4. Listar Usuarios
5. Listar Editoria